# Assignment 1 - Eleanor Adachi

## 1. Admin

We created a fork of the main GitHub repository. Our code can be found here: https://github.com/eleanor-adachi/ARE212_Materials

## 2. Exercises

### (1)

**From ARE210, recall (Section 9 in Mahajan’s “Handout 1”) the rule for computing the distribution of certain transformations
of random variables (the “inverse Jacobian rule”). Let** $(x, y)$ **be independently distributed continuous random variables possessing densities** $f_x$ **and** $f_y$. **Let** $z = x+y$. **Use the rule to obtain an expression for the distribution of** $z$.

Helpful: https://andymiller.github.io/2015/08/09/integral-jacobian.html

Helpful: https://stats.libretexts.org/Bookshelves/Probability_Theory/Probability_Mathematical_Statistics_and_Stochastic_Processes_(Siegrist)/03%3A_Distributions/3.07%3A_Transformations_of_Random_Variables

Given two random variables $x$ and $y$ where $y=f(x)$ and $x$ has probability density function $p_x(x)$

Then $p_y(y) = p_x(x)|\frac{dx}{dy}| = p_x(x)|\frac{df^{-1}(y)}{dy}|$

For $z = x + y$ where $x$ has probability density function $f_x(x)$ and $y$ has probability density function $f_y(y)$,

$f_z(z) = f_x(x)|\frac{\partial x}{\partial z}| \cdot f_y(y)|\frac{\partial y}{\partial z}|$

$x = z - y$ and $y = z - x$, so $\frac{\partial x}{\partial z} = 1$ and $\frac{\partial y}{\partial z} = 1$

Therefore, $z$ has a continuous distribution with probability density $f_z$ given by

$f_z(z) = \int_{-\infty}^{\infty} f_x(x) f_y(z-x) \,dx$

## 3. Convolutions

**We’ve discussed ways to program a convolution of random variables in a Jupyter notebook** [ipynb](https://github.com/ligonteaching/ARE212_Materials/blob/master/random_variables0.ipynb).

### (1)

**As in the notebook, consider a discrete random variable** $s$ **and a continuous random variable** $x$. **Prove that the convolution of** $s$ **and** $x$ **(or, informally,** $x+s$**) has a continuous distribution, as suggested by the figure at the end of the notebook, or establish that the figure is wrong or misleading.**

Let $z = x + s$, $f_x(x)$ be the probability distribution function of $x$ and $f_s(s)$ be the probability distribution function of $s$

$f_z(z) = \int_{-\infty}^{\infty} f_x(z-s) f_s(s) \,ds$

If $f_s(s)$ is discrete, then $f_s(s) = \sum_{n}^{} f_{s,n} \delta(s - s_n)$ where $f_{s,n}$ is the probability of $s_n$

Therefore $f_z(z) = \sum_{n}^{} f_x(z - s_n) f_{s,n}$

Since this is a sum of continuous functions, $f_z(z)$ is also a continuous function

### (2)

**The notebook develops a simple class** `ConvolvedContinuousAndDiscrete` **to allow for the creation and manipulations of (you guessed it) convolutions of a continuous rv with a discrete rv. Can you develop a similar class for convolutions of independent discrete random variables?**

In [1]:
from scipy.stats import distributions as iid

class ConvolvedTwoDiscrete(iid.rv_discrete):

    """Convolve (add) a discrete rv s1 and another discrete rv s2,
       returning the resulting cdf."""

    def __init__(self,s1,s2):
        self.discrete_rv1 = s1
        self.discrete_rv2 = s2
        super(ConvolvedTwoDiscrete, self).__init__(name="ConvolvedTwoDiscrete")
        
    def _cdf(self,z):
        F=0
        s1 = self.discrete_rv1
        s2 = self.discrete_rv2
        
        for k in range(len(s2.xk)):
            F = F + s1.cdf(z-s2.xk[k])*s2.pk[k] # convolution
        return F

    def _pmf(self,z):
        f=0
        s1 = self.discrete_rv1
        s2 = self.discrete_rv2
        
        for k in range(len(s2.xk)):
            f = f + s1.pmf(z-s2.xk[k])*s2.pk[k]
        return f

In [13]:
Omega1 = (-1,0,1)
Pr1 = (1/3.,1/2.,1/6.) # add up to 1

s1 = iid.rv_discrete(values=(Omega1,Pr1))

Omega2 = (-1,1)
Pr2 = (1/2.,1/2) # add up to 1

s2 = iid.rv_discrete(values=(Omega2,Pr2))

# match Neri
s1 = iid.rv_discrete(values=([-4, 0, 4], [0.1, 0.2, 0.7]))
s2 = iid.rv_discrete(values=([1, 2, 3], [0.4, 0.3, 0.3]))

In [14]:
y = ConvolvedTwoDiscrete(s1,s2)

In [16]:
import plotly.graph_objects as go
import numpy as np

X = np.linspace(-1.1,2.1,33).tolist()

# match Neri
X = np.linspace(min(s1.a, s2.a) + min(s1.xk + s2.xk), max(s1.b, s2.b) + max(s1.xk + s2.xk), 100)

fig = go.Figure(data=go.Scatter(x=X, y=[y.pmf(z) for z in X]))
fig.show()

In [5]:
fig = go.Figure(data=go.Scatter(x=X, y=[y.cdf(z) for z in X]))
fig.show()

### (3)

**Same as (2), but convolutions of independent continuous random variables?**

Helpful: https://stats.libretexts.org/Bookshelves/Probability_Theory/Probability_Mathematical_Statistics_and_Stochastic_Processes_(Siegrist)/03%3A_Distributions/3.07%3A_Transformations_of_Random_Variables

Helpful for CDF: https://web.stanford.edu/class/archive/cs/cs109/cs109.1182/lectures/15%20Convolution.pdf

In [6]:
from scipy.stats import distributions as iid
from scipy.integrate import quad
import numpy as np

class ConvolvedTwoContinuous(iid.rv_continuous):

    """Convolve (add) a continuous rv x1 and another continuous rv x2,
       returning the resulting cdf."""

    def __init__(self,x1,x2):
        self.continuous_rv1 = x1
        self.continuous_rv2 = x2
        super(ConvolvedTwoContinuous, self).__init__(name="ConvolvedTwoContinuous")
        
    def _cdf(self,z):
        F=0
        x1 = self.continuous_rv1
        x2 = self.continuous_rv2
        
        # use CDF of x1
        F = quad(lambda x: x1.cdf(x)*x2.pdf(z-x), -np.inf, np.inf)
        
        return F

    def _pdf(self,z):
        f=0
        x1 = self.continuous_rv1
        x2 = self.continuous_rv2
        
        f = quad(lambda x: x1.pdf(x)*x2.pdf(z-x), -np.inf, np.inf)
        
        return f

In [17]:
x1 = iid.norm()
x2 = iid.norm(loc=1, scale=2)

# match Neri
x2 = iid.norm()

In [18]:
y = ConvolvedTwoContinuous(x1,x2)

In [20]:
X = np.linspace(-10,10,500).tolist()

# match Neri
X = np.linspace(-4, 10, 100).tolist()

fig = go.Figure(data=go.Scatter(x=X, y=[y.pdf(z) for z in X]))
fig.show()

In [21]:
fig = go.Figure(data=go.Scatter(x=X, y=[y.cdf(z) for z in X]))
fig.show()

## 5. Simultaneous Equations

**When we defined the general weighted regression, we didn’t assume anything about the dimension of the different objects except that they were 'conformable.'**

**So: consider**

(2) $y = X\beta + u$; **with** $E[T'u] = 0$, **and where** $y = [y_1, y_2, ... , y_k]$, **so that if you had a sample of** $N$ **observations realizations of** $y$ **would be an** $N \times k$ **matrix.**

### (1)

**What does our assumption of conformability then imply about the dimensions of** $X, \beta, T$, **and** $u$?

Our assumption of conformability implies that $X$ is $N \times \ell$, $\beta$ is $\ell \times k$, $T$ is $N \times \ell$, and $u$ is $N \times k$.

### (2)

**Could you use the estimator we developed in** `weighted_regression.ipynb` **to estimate this system of simultaneous equations?**

No, because that estimator assumes that $y$ only has one column.

### (3)

**Extend the code in** `weighted_regression.ipynb` **to actually estimate** $\beta$ **in the case with** $k = 3$.

To construct a sample of observables $(y,X,T)$:

In [11]:
# NOTE copied from weighted_regression.ipynb

%matplotlib inline
import numpy as np
from scipy.stats import multivariate_normal

k = 3 # number of columns in y
l = 4 # number of columns in X
N = 100 # sample size

mu = [0]*l
Sigma = [
    [1,0.5,0,0.2],
    [0.5,2,0,0],
    [0,0,3,0],
    [0.2,0,0,1.5]]
T = multivariate_normal(mu,Sigma)

u = multivariate_normal([0]*k, 0.5)

beta = np.array(
    [[0.5, 0.7, 0.9],
     [1, 1.2, 1.4],
     [2, 2.2, 2.4],
     [3, 3.2, 3.4]])

D = np.random.random(size=(N, l)) # Generate random matrix

# Now: Transform rvs into a sample
T = T.rvs(N)

u = u.rvs(N) # Replace u with a sample

X = (T**3)*D  # element-wise multiplication

y = X@beta + u # Note use of @ operator for matrix multiplication

In the classical case we were trying to solve a linear system that took the form $Ab=0$, with $A$ a square matrix.  In the present case we're also trying to solve a linear system, but with a matrix $A$ that may have more rows than columns.  Provided the rows are linearly independent, this implies that we have an **overidentified** system of equations.  We'll return to the implications of this later, but for now this also calls for a different numerical approach, using `np.linalg.lstsq` instead of `np.linalg.solve`.

In addition, since $y$ is $N \times k$, we need to use the full formula to solve for $\hat{\beta}$:

$\hat{\beta} = (X'TT'X)^{-1} (X'TT'y)$

In [12]:
from scipy.linalg import inv, sqrtm

b = np.linalg.lstsq((X.T@T)@(T.T@X), (X.T@T)@(T.T@y), rcond=None)[0] # lstsq returns several results

print(b)

[[0.50655627 0.72736329 0.95630646]
 [1.0082531  1.18841854 1.37477634]
 [2.00242854 2.19153483 2.39196434]
 [3.01785328 3.20356753 3.37545492]]


### (4)

**What additional assumptions are necessary to estimate the distribution of the estimator of** $\beta$?

We must make an assumption about homoskedasticity or heteroskedasticity of $u$ in order to estimate the variance-covariance matrix and standard errors for our estimate of $\beta$.

## 8. “Plug-in” Kernel Bias Estimator

**In our discussion of bias of the kernel density estimator in lecture we constructed an “Oracle” estimator, which can be implemented when we know the true density** $f$ **that we’re trying to estimate.**

**Of course, the Oracle estimator is only feasible when we don’t need it. What about the idea of using the same expression for bias as in the Oracle case, but replacing** $f$ **with our estimate** $\hat{f}$? **Would this tell us anything useful? If so, under what conditions? What pitfalls might one encounter?**

Helpful: https://www.stat.berkeley.edu/~stark/Teach/S240/Notes/ch10.pdf

If $f$ itself is very smooth, then replacing $f$ with $\hat{f}$ in the Oracle estimator would still give you useful information about bandwidth selection. However, the bias estimate would be inaccurate for parts of the distribution where $f$ is changing more rapidly. 